# Import necessary packages

In [37]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

import xgboost as xgb

from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV

from timeit import default_timer as timer

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import original data and preprocessed data

In [39]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/calendar.csv')

**Note: This Step is for the extraction of California State and Foods category**

In [40]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'FOODS')]

In [41]:
sales_train_val

id      item_id  dept_id cat_id store_id  \
1612   FOODS_1_001_CA_1_validation  FOODS_1_001  FOODS_1  FOODS     CA_1   
1613   FOODS_1_002_CA_1_validation  FOODS_1_002  FOODS_1  FOODS     CA_1   
1614   FOODS_1_003_CA_1_validation  FOODS_1_003  FOODS_1  FOODS     CA_1   
1615   FOODS_1_004_CA_1_validation  FOODS_1_004  FOODS_1  FOODS     CA_1   
1616   FOODS_1_005_CA_1_validation  FOODS_1_005  FOODS_1  FOODS     CA_1   
...                            ...          ...      ...    ...      ...   
12191  FOODS_3_823_CA_4_validation  FOODS_3_823  FOODS_3  FOODS     CA_4   
12192  FOODS_3_824_CA_4_validation  FOODS_3_824  FOODS_3  FOODS     CA_4   
12193  FOODS_3_825_CA_4_validation  FOODS_3_825  FOODS_3  FOODS     CA_4   
12194  FOODS_3_826_CA_4_validation  FOODS_3_826  FOODS_3  FOODS     CA_4   
12195  FOODS_3_827_CA_4_validation  FOODS_3_827  FOODS_3  FOODS     CA_4   

      state_id  d_1  d_2  d_3  d_4  ...  d_1904  d_1905  d_1906  d_1907  \
1612        CA    3    0    0    1  ...       0       2       0       4   
1613        CA    0    1    0    1  ...       1       3       1       0   
1614        CA    0    0    0    0  ...       3       0       2       1   
1615        CA    0    0    0    0  ...       0       0       0       0   
1616        CA    3    9    3    3  ...       3       1       1       2   
...        ...  ...  ...  ...  ...  ...     ...     ...     ...     ...   
12191       CA    1    1    1    4  ...       3       1       2       2   
12192       CA    0    0    0    1  ...       0       1       1       1   
12193       CA    2    3    2    1  ...       0       2       0       1   
12194       CA    0    0    0    0  ...       1       2       2       1   
12195       CA    0    0    0    0  ...       0       0       0       0   

       d_1908  d_1909  d_1910  d_1911  d_1912  d_1913  
1612        1       1       0       1       1       0  
1613        0       1       2       0       0       0  
1614        1       0       1       0       1       0  
1615        0       0       0       0       0       0  
1616        0       2       2       1       4       1  
...       ...     ...     ...     ...     ...     ...  
12191       0       1       1       3       3       0  
12192       0       2       0       0       0       1  
12193       3       1       1       0       2       0  
12194       4       5       1       7       0       4  
12195       2       0       4       4       4       8  

[5748 rows x 1919 columns]

In [42]:
INPUT_DIR_2 = r'/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/List of Product ID according to 4 demand patterns/California/Foods'

list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [43]:
len(sales_intermittent)

1301

In [44]:
len(sales_lumpy)

395

In [45]:
len(sales_erratic)

458

In [46]:
len(sales_smooth)

3594

**Check by looking at column "id" to verify if we choose the right product (FOODS or HOBBIES or HOUSEHOLD)**

In [47]:
sales_intermittent

id      item_id  dept_id cat_id store_id  \
1618   FOODS_1_008_CA_1_validation  FOODS_1_008  FOODS_1  FOODS     CA_1   
1619   FOODS_1_009_CA_1_validation  FOODS_1_009  FOODS_1  FOODS     CA_1   
1620   FOODS_1_010_CA_1_validation  FOODS_1_010  FOODS_1  FOODS     CA_1   
1621   FOODS_1_011_CA_1_validation  FOODS_1_011  FOODS_1  FOODS     CA_1   
1627   FOODS_1_017_CA_1_validation  FOODS_1_017  FOODS_1  FOODS     CA_1   
...                            ...          ...      ...    ...      ...   
12174  FOODS_3_806_CA_4_validation  FOODS_3_806  FOODS_3  FOODS     CA_4   
12189  FOODS_3_821_CA_4_validation  FOODS_3_821  FOODS_3  FOODS     CA_4   
12191  FOODS_3_823_CA_4_validation  FOODS_3_823  FOODS_3  FOODS     CA_4   
12192  FOODS_3_824_CA_4_validation  FOODS_3_824  FOODS_3  FOODS     CA_4   
12193  FOODS_3_825_CA_4_validation  FOODS_3_825  FOODS_3  FOODS     CA_4   

      state_id  d_1  d_2  d_3  d_4  ...  d_1904  d_1905  d_1906  d_1907  \
1618        CA    0    0    0    0  ...       0       1       0       0   
1619        CA    0    0    0    0  ...       0       0       0       0   
1620        CA    0    0    0    0  ...       0       0       0       0   
1621        CA    2    1    0    0  ...       0       2       0       1   
1627        CA    0    0    0    0  ...       2       2       0       1   
...        ...  ...  ...  ...  ...  ...     ...     ...     ...     ...   
12174       CA    0    0    0    0  ...       0       0       0       0   
12189       CA    0    0    0    0  ...       0       0       0       0   
12191       CA    1    1    1    4  ...       3       1       2       2   
12192       CA    0    0    0    1  ...       0       1       1       1   
12193       CA    2    3    2    1  ...       0       2       0       1   

       d_1908  d_1909  d_1910  d_1911  d_1912  d_1913  
1618        0       0       0       0       0       0  
1619        0       0       0       0       0       0  
1620        0       0       0       0       2       0  
1621        3       2       3       1       0       1  
1627        0       0       1       2       1       4  
...       ...     ...     ...     ...     ...     ...  
12174       0       0       1       0       0       1  
12189       1       1       0       3       0       0  
12191       0       1       1       3       3       0  
12192       0       2       0       0       0       1  
12193       3       1       1       0       2       0  

[1301 rows x 1919 columns]

# User-defined functions to calculate Metrics

In [48]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [49]:
sales_pattern = sales_intermittent.copy()

In [50]:
sales_pattern_py = sales_pattern.copy()

list_pattern_py = sales_pattern_py.id.unique().tolist()
list_pattern_py = [(re.sub(r'_validation', r"", x)) for x in list_pattern_py]

sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
df_pattern.drop('Date', axis=1, inplace=True)
df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
df_pattern.index.names = ['Date']
df_pattern = df_pattern.astype('float64')

df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
df_pattern.reset_index(inplace=True)

calendar['Date'] = pd.to_datetime(calendar['date'])

dataframe = pd.merge(df_pattern,
                     calendar[['wm_yr_wk', 'Date']],
                     on = 'Date')

not_enough_seven_days = dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})[dataframe.groupby('wm_yr_wk').agg({'Date': 'count'})['Date'] < 7].index.tolist()
dataframe = dataframe[~dataframe['wm_yr_wk'].isin(not_enough_seven_days)]

dataframe.set_index('wm_yr_wk', inplace=True)
dataframe.drop('Date', axis=1, inplace=True)

dataframe_weekly = dataframe.groupby('wm_yr_wk').sum().reset_index()

In [51]:
dataframe_weekly

wm_yr_wk  FOODS_1_005_CA_2  FOODS_1_005_CA_4  FOODS_1_008_CA_1  \
0       11101              36.0               4.0               0.0   
1       11102              37.0              13.0               0.0   
2       11103              40.0               3.0               0.0   
3       11104              30.0               1.0               0.0   
4       11105              33.0               8.0               0.0   
..        ...               ...               ...               ...   
268     11608              38.0               6.0               0.0   
269     11609              19.0              13.0               0.0   
270     11610              29.0               7.0               0.0   
271     11611              43.0               7.0               0.0   
272     11612              22.0              11.0               0.0   

     FOODS_1_008_CA_4  FOODS_1_009_CA_1  FOODS_1_009_CA_2  FOODS_1_009_CA_3  \
0                 1.0               4.0               6.0               6.0   
1                 3.0               5.0              18.0              19.0   
2                 3.0               0.0               8.0               4.0   
3                 8.0               0.0               0.0               0.0   
4                 3.0               0.0               0.0               0.0   
..                ...               ...               ...               ...   
268               1.0               2.0               5.0              16.0   
269               1.0               4.0              18.0               9.0   
270               0.0              10.0               3.0              13.0   
271               2.0               5.0               6.0               7.0   
272               5.0               7.0              10.0               8.0   

     FOODS_1_009_CA_4  FOODS_1_010_CA_1  ...  FOODS_3_821_CA_4  \
0                 5.0               0.0  ...               0.0   
1                 5.0               0.0  ...               0.0   
2                 2.0               0.0  ...               0.0   
3                 0.0               0.0  ...               0.0   
4                 0.0               0.0  ...               0.0   
..                ...               ...  ...               ...   
268               6.0               3.0  ...               0.0   
269               0.0               0.0  ...               0.0   
270               0.0               2.0  ...               0.0   
271               3.0               0.0  ...               0.0   
272               7.0               4.0  ...               0.0   

     FOODS_3_823_CA_1  FOODS_3_823_CA_2  FOODS_3_823_CA_3  FOODS_3_823_CA_4  \
0                 0.0              24.0              18.0               2.0   
1                 0.0              30.0               6.0               2.0   
2                16.0               8.0              31.0              20.0   
3                 6.0               3.0              20.0               7.0   
4                 6.0               0.0               2.0               0.0   
..                ...               ...               ...               ...   
268               0.0              17.0               0.0               3.0   
269               0.0              19.0               0.0               0.0   
270               0.0              17.0               0.0               1.0   
271               0.0              31.0               2.0               2.0   
272               0.0              26.0               0.0               0.0   

     FOODS_3_824_CA_1  FOODS_3_824_CA_2  FOODS_3_824_CA_4  FOODS_3_825_CA_4  \
0                 2.0               3.0               1.0               6.0   
1                 1.0               0.0               0.0               0.0   
2                 0.0               0.0               0.0               0.0   
3                 3.0               1.0               0.0               3.0   
4                 0.0               0.0               0.0         

In [52]:
lags = list(range(1, 25))
moving_averages = list(range(1, 25)) 
stds = list(range(2, 25))
maximums = list(range(1, 25))
minimums = list(range(1, 25))
totals = list(range(1, 25))
num_days_zeros = list(range(1, 25))

correlation_threshold = 0.8

end_train_week = 11610

num_id_sample_for_tuning = 200

In [53]:
sample_list = sample(list_pattern_py, num_id_sample_for_tuning)

In [54]:
dataframe_check = dataframe_weekly[sample_list].iloc[-2:].T

dataframe_check['Check'] = np.where((dataframe_check[271] == 0) &
                                    (dataframe_check[272] == 0), 1, 0)

sample_removed = dataframe_check[dataframe_check['Check'] == 1].index.tolist()

In [55]:
sample_list_filtered = [x for x in sample_list if (x not in sample_removed)]

In [56]:
len(sample_list_filtered)

113

In [57]:
start = timer()

df_output_hyper_params = pd.DataFrame()
    
for product in sample_list_filtered:
    print('Currently Running: %s' % product)
    print('Progressing: {0} %'.format(round(sample_list_filtered.index(product) / len(sample_list_filtered) * 100, 2)))

    dataframe_product = dataframe_weekly[['wm_yr_wk', product]]

    # Create Lag variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (t-{lag})': dataframe_product[product].shift(lag)
        for lag in lags
    })

    # Create Rolling Moving Average variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
        for moving_average in moving_averages
    })

    # Create Rolling Standard Deviation variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
        for std in stds
    })

    # Create Rolling Maximum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
        for maximum in maximums
    })

    # Create Rolling Minimum variables
    dataframe_product = dataframe_product.assign(**{
        f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
        for minimum in minimums
    })

    # Create Rolling Total variables 
    dataframe_product = dataframe_product.assign(**{
        f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
        for total in totals
    })

    # Create Zero Count variables
    count_func = lambda x: (x==0).sum()

    dataframe_product = dataframe_product.assign(**{
        f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
        for num_days_zero in num_days_zeros
    })

    dataframe_product.dropna(inplace=True)
    dataframe_product.set_index('wm_yr_wk', inplace=True)

    train_data = dataframe_product[dataframe_product.index <= end_train_week]

    corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
    high_corr_var=np.where(corr_matrix > correlation_threshold)
    high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

    train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

    X_train = train_data.drop(product, axis=1)
    y_train = train_data[product]  

    tscv = TimeSeriesSplit(n_splits=3)

    params = { 'max_depth': [3, 5, 7, 10],
              'learning_rate': [0.01, 0.1, 0.2, 0.3],
              'subsample': np.arange(0.5, 1.0, 0.1),
              'colsample_bytree': np.arange(0.5, 1.0, 0.1),
              'colsample_bylevel': np.arange(0.5, 1.0, 0.1),
              'n_estimators': [30, 50, 100]}

    xgbr = xgb.XGBRegressor(seed = 20)

    regressor = RandomizedSearchCV(estimator=xgbr,
                                  param_distributions=params,
                                  cv = tscv, 
                                  scoring='neg_mean_absolute_percentage_error',
                                  n_iter=300,
                                  verbose=1)
    regressor.fit(X_train, y_train)
    best_params_temp = regressor.best_params_

    df_output_temp = pd.DataFrame.from_dict(best_params_temp, orient='index').T
    df_output_temp['Product'] = product

    df_output_hyper_params = df_output_hyper_params.append(df_output_temp, ignore_index=True)

    
end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

Streaming output truncated to the last 5000 lines.
[13:15:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:15:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:15:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:15:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:15:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:15:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:15:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[13:15:18] WARNING: /workspace/src/objective/regression_obj.

In [58]:
df_output_hyper_params.to_csv('/content/drive/MyDrive/Colab Notebooks_Weekly Prediction/XGBoost (Completed Pipeline)/Foods/Intermittent_Params_Tuning_XGBoost.csv')

# Decide optimum set of parameters for XGBoost algorithm

In [59]:
# See if any values larger than 1 in column "Product"
# If yes, the corresponding set of parameters occur more than once and those sets should be considered first
df_output_hyper_params.groupby(['subsample', 'n_estimators', 'max_depth', 
                         'learning_rate', 'colsample_bytree',
                         'colsample_bylevel'])[['Product']].count().sort_values(by = 'Product', ascending=False)

Product
subsample n_estimators max_depth learning_rate colsample_bytree colsample_bylevel         
0.7       30.0         10.0      0.01          0.9              0.9                      3
0.8       30.0         10.0      0.01          0.9              0.9                      2
0.9       30.0         10.0      0.01          0.7              0.7                      2
0.8       30.0         10.0      0.01          0.8              0.6                      2
                                               0.6              0.7                      2
...                                                                                    ...
0.6       30.0         7.0       0.01          0.9              0.8                      1
                                                                0.6                      1
                                               0.7              0.6                      1
                                               0.5              0.5                      1
0.9       100.0        10.0      0.30          0.6              0.5                      1

[100 rows x 1 columns]

In [60]:
# See what value of parameter "subsample" occurs the most frequently
df_output_hyper_params.groupby(['subsample'])[['Product']].count()

Product
subsample         
0.5             22
0.6             24
0.7             18
0.8             23
0.9             26

In [61]:
# See what value of parameter "n_estimators" occurs the most frequently
df_output_hyper_params.groupby(['n_estimators'])[['Product']].count()

Product
n_estimators         
30.0              106
50.0                3
100.0               4

In [62]:
# See what value of parameter "max_depth" occurs the most frequently
df_output_hyper_params.groupby(['max_depth'])[['Product']].count()

Product
max_depth         
3.0              7
5.0             11
7.0             43
10.0            52

In [63]:
# See what value of parameter "learning_rate" occurs the most frequently
df_output_hyper_params.groupby(['learning_rate'])[['Product']].count()

Product
learning_rate         
0.01                98
0.10                 5
0.20                 1
0.30                 9

In [64]:
# See what value of parameter "colsample_bytree" occurs the most frequently
df_output_hyper_params.groupby(['colsample_bytree'])[['Product']].count()

Product
colsample_bytree         
0.5                     9
0.6                    16
0.7                    17
0.8                    31
0.9                    40

In [65]:
# See what value of parameter "colsample_bytree" occurs the most frequently
df_output_hyper_params.groupby(['colsample_bylevel'])[['Product']].count()

Product
colsample_bylevel         
0.5                     13
0.6                     24
0.7                     24
0.8                     22
0.9                     30

# Implement XGBoost on Test Set with optimum set of parameters

### Specify optimum set of parameters

In [66]:
subsample_chosen = 0.7
n_estimators_chosen = 30
max_depth_chosen = 10
learning_rate_chosen = 0.1               
colsample_bytree_chosen =  0.9
colsample_bylevel_chosen = 0.9

In [68]:
len(list_pattern_py)

1301

In [69]:
list_pattern_temp = list_pattern_py.copy()

In [74]:
df_output_xgb_pattern = pd.DataFrame()
for product in list_pattern_temp:
  print('Currently Running: %s' % product)
  print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))
  dataframe_product = dataframe_weekly[['wm_yr_wk', product]]
  # Create Lag variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (t-{lag})': dataframe_product[product].shift(lag)
      for lag in lags
  })

  # Create Rolling Moving Average variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
      for moving_average in moving_averages
  })

  # Create Rolling Standard Deviation variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
      for std in stds
  })

  # Create Rolling Maximum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
      for maximum in maximums
  })

  # Create Rolling Minimum variables
  dataframe_product = dataframe_product.assign(**{
      f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
      for minimum in minimums
  })

  # Create Rolling Total variables 
  dataframe_product = dataframe_product.assign(**{
      f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
      for total in totals
  })

  dataframe_product.dropna(inplace=True)
  dataframe_product.set_index('wm_yr_wk', inplace=True)

  train_data = dataframe_product[dataframe_product.index <= end_train_week]
  test_data = dataframe_product[dataframe_product.index > end_train_week]

  corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
  high_corr_var=np.where(corr_matrix > correlation_threshold)
  high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

  train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
  test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)

  X_train = train_data.drop(product, axis=1)
  y_train = train_data[product]

  X_test = test_data.drop(product, axis=1)
  y_test = test_data[product]

  xgb_r = xgb.XGBRegressor(seed = 20,
                          subsample = subsample_chosen,
                          n_estimators = n_estimators_chosen,
                          max_depth = max_depth_chosen,
                          learning_rate = learning_rate_chosen,               
                          colsample_bytree = colsample_bytree_chosen,
                          colsample_bylevel = colsample_bylevel_chosen)
  

  xgb_r.fit(X_train,y_train)
  y_pred = xgb_r.predict(X_test)
  df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                  'Forecast': y_pred, 
                                  'Product': [product for num_rows in range(len(y_pred))]
                                  })
  
  df_output_xgb_pattern = df_output_xgb_pattern.append(df_output_temp, ignore_index=False)  

Currently Running: FOODS_1_008_CA_1
Progressing: 0.0 %
[13:32:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running: FOODS_1_009_CA_1
Progressing: 0.08 %
[13:32:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running: FOODS_1_010_CA_1
Progressing: 0.15 %
[13:32:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running: FOODS_1_011_CA_1
Progressing: 0.23 %
[13:32:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running: FOODS_1_017_CA_1
Progressing: 0.31 %
[13:32:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running: FOODS_1_019_CA_1
Progressing: 0.38 %
[13:32:34] WARNING: /workspace

In [76]:
df_output_xgb_pattern

Actual Data  Forecast           Product
wm_yr_wk                                         
11611             0.0  0.001662  FOODS_1_008_CA_1
11612             0.0  0.001662  FOODS_1_008_CA_1
11611             5.0  5.467693  FOODS_1_009_CA_1
11612             7.0  5.040196  FOODS_1_009_CA_1
11611             0.0  1.263218  FOODS_1_010_CA_1
...               ...       ...               ...
11612             0.0  0.255400  FOODS_3_823_CA_4
11611             1.0  0.435732  FOODS_3_824_CA_4
11612             0.0  0.209042  FOODS_3_824_CA_4
11611             1.0  2.880284  FOODS_3_825_CA_4
11612             1.0  2.865216  FOODS_3_825_CA_4

[2602 rows x 3 columns]

In [82]:
df_output_xgb_pattern.to_csv('Intermittent_XGBoost_Forecast_Values.csv')

In [77]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [79]:
# Start calculating metrics 
start = timer()

df_forecast_metrics = df_output_xgb_pattern.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.060970078600000005 minutes


In [80]:
df_forecast_metrics

Product      MASE     WMAPE     SMAPE      MAPE        MAE  \
0     FOODS_1_005_CA_2  0.718213   46.4076   48.7073   47.2085  15.082464   
1     FOODS_1_005_CA_4  0.881304   39.1691   46.1569   36.8135   3.525216   
2     FOODS_1_008_CA_1       inf       inf  200.0000       NaN   0.001662   
3     FOODS_1_008_CA_4  0.463727   39.7480   44.2353   35.6734   1.391181   
4     FOODS_1_009_CA_1  0.606874   20.2291   20.7451   18.6755   1.213748   
...                ...       ...       ...       ...       ...        ...   
1296  FOODS_3_824_CA_1  1.155694  231.1387  127.9189   77.4651   2.311387   
1297  FOODS_3_824_CA_2  1.649276  329.8553  111.9236   27.0755   1.649276   
1298  FOODS_3_824_CA_4  0.386655   77.3310  139.3018   56.4268   0.386655   
1299  FOODS_3_825_CA_4       inf  187.2750   96.7138  187.2750   1.872750   
1300  FOODS_3_826_CA_3       inf       inf  200.0000       NaN   0.031739   

           RMSE  
0     15.643316  
1      3.909499  
2      0.001662  
3      1.639684  
4      1.424705  
...         ...  
1296   2.433780  
1297   2.149519  
1298   0.425498  
1299   1.872765  
1300   0.031989  

[1301 rows x 7 columns]

In [81]:
df_forecast_metrics.to_csv('XGBoost_Intermittent_Forecast_Metrics.csv')